Google Classroomの成績情報をCSV出力するコード

In [5]:
import os
import csv
import pickle
import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

class GC_Score2CSV:

  def __init__(self, client_secret_json):
    self.CLIENT_SECRETS_FILE = client_secret_json
    self.SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', 'https://www.googleapis.com/auth/classroom.student-submissions.students.readonly', 'https://www.googleapis.com/auth/classroom.rosters.readonly', 'https://www.googleapis.com/auth/classroom.profile.emails']
    self.API_SERVICE_NAME = 'classroom'
    self.API_VERSION = 'v1'
    self.service = self.get_authenticated_service()
    self.courseId = None

  # Google Auth Init
  def get_authenticated_service(self):
    if os.path.exists("CREDENTIALS_PICKLE_FILE"):
        with open("CREDENTIALS_PICKLE_FILE", 'rb') as f:
            credentials = pickle.load(f)
    else:
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(self.CLIENT_SECRETS_FILE, self.SCOPES)
        credentials = flow.run_console()
        with open("CREDENTIALS_PICKLE_FILE", 'wb') as f:
            pickle.dump(credentials, f)
    return googleapiclient.discovery.build(self.API_SERVICE_NAME, self.API_VERSION, credentials=credentials)

  # Get course info
  def get_course_info(self, nextPageToken=None, output=[]):
    request = self.service.courses().list(
        pageToken=nextPageToken
    ).execute()
    for info in request["courses"]:
      output.append(info)
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return get_course_info(nextPageToken, output)

  # Set course ID
  def set_target_course(self, courseinfo=None):
    if courseinfo == None:
      courseinfo = self.get_course_info()
    print("Course lists is shown below: \n")
    for i in range(len(courseinfo)):
      print(str(i + 1) + ": " + courseinfo[i]["name"] + " (" + courseinfo[i]["alternateLink"] + ")")
    id = 0
    print("")
    while True:
      if id in range(1, len(courseinfo) + 1):
        self.courseId = courseinfo[id - 1]["id"]
        break
      else:
        id = int(input("Enter course ID: "))

  # Get students info
  def get_students_info(self, courseId, nextPageToken=None, output=[]):
    request = self.service.courses().students().list(
        courseId=courseId,
        pageToken=nextPageToken
    ).execute()
    for info in request["students"]:
      output.append(info)
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_students_info(courseId, nextPageToken, output)

  # Get assignments info
  def get_assignments_info(self, courseId, nextPageToken=None, output=[]):
    request = self.service.courses().courseWork().list(
        courseId=courseId,
        pageToken=nextPageToken
    ).execute()
    for info in request["courseWork"]:
      output.append(info)
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_assignments_info(courseId, nextPageToken, output)


  # Get all of the submitted assignments info
  def get_submitted_assignments_all_info(self, courseId, nextPageToken=None, output=[]):
    request = self.service.courses().courseWork().studentSubmissions().list(
        courseId=courseId,
        courseWorkId="-",
        pageToken=nextPageToken
    ).execute()
    for info in request["studentSubmissions"]:
      output.append(info)
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_submitted_assignments_all_info(courseId, nextPageToken, output)

  # Get submitted assignments info for specific assignment
  def get_submitted_assignments_info(self, courseId, assignmentId, assignments_all_info=None):
    if assignments_all_info == None:
      assignments_all_info = self.get_submitted_assignments_all_info(self.courseId)
    output = []
    for info in assignments_all_info:
      if info["courseWorkId"] == assignmentId:
        output.append(info)
    return output

  # Get score of assignments for each student
  def get_submitted_assignments_score(self, courseId, assignmentId, assignments_info=None, assignments_all_info=None):
    if assignments_info == None:
      assignments_info = self.get_submitted_assignments_info(courseId, assignmentId, assignments_all_info)
    output = {}
    for info in assignments_info:
      try:
        # 学生に返却されている点数を出力した場合はassignedGradeを有効化、学生に返却されていない仮の点数を出力したい場合はdraftGradeをコメントアウト解除し実行する
        # output[info["userId"]] = info["assignedGrade"]
        output[info["userId"]] = info["draftGrade"]
      except:
        pass
    return output

  # CSV Export
  def export_score_data(self, courseId=None, exportPath="./export_score.csv"):
    if courseId == None:
      courseId = self.courseId

    studentsinfo = self.get_students_info(self.courseId)
    assignmentsinfo = self.get_assignments_info(self.courseId)
    output = []
    assignments_allinfo = self.get_submitted_assignments_all_info(self.courseId)
    for info in assignmentsinfo:
      output_list = self.get_submitted_assignments_score(self.courseId, info["id"], assignments_all_info=assignments_allinfo)
      output_list['info'] = {'id': info["id"], 'title': info.get("title"), 'max': info.get("maxPoints")}
      output.append(output_list)

    final_output = []
    final_output.append([info.get("info")["title"] for info in output])
    final_output[0].insert(0, "EmailAddress")
    final_output.append([info.get("info")["max"] for info in output])
    final_output[1].insert(0, "max_points")
    for user in studentsinfo:
      b = [info.get(user["profile"]["id"]) for info in output]
      b.insert(0, user["profile"]["emailAddress"])
      final_output.append(b)

    with open(exportPath, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(final_output)

GoogleのAPIを有効化するために、Google API Consoleからプロジェクトを作成し、Oauthクライアントを作成 (JSONで認証情報をダウンロードしてくる)すると同時に、Google Classroom APIを有効化しておく

In [6]:
score2csv = GC_Score2CSV("./client_secret_hogehoge.json")

成績情報を取得したいクラスを選択する

In [ ]:
score2csv.set_target_course()

これを実行することによって、Google ClassroomのGradeページのような形で成績情報がダウンロードできる

In [8]:
score2csv.export_score_data()